<a href="https://colab.research.google.com/github/AbdalazizOth/Tradtional_dict/blob/main/Ubab_Zakhir.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**[UbabZakhir_dict](https://github.com/OpenITI/0650AH/tree/master/data/0650IbnMuhammadRadiDinSaghani/0650IbnMuhammadRadiDinSaghani.CubabZakhir)**